In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [64]:
#ccba
ccba = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_cdtx0001_full_hashed.csv')
#custinfo
cdtx = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_custinfo_full_hashed.csv')
#dp
dp = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_remit1_full_hashed.csv')
#alert_time
alert_time = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_x_alert_date.csv')
#y
y = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/train_y_answer.csv')
#案件名單
doc = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/預測的案件名單及提交檔案範例.csv')

In [65]:
doc_merged = cdtx.merge(doc, on='alert_key', how='right').merge(alert_time, on='alert_key', how='left')
doc_merged = doc_merged[~doc_merged['cust_id'].isnull()]
max_doc = doc_merged.groupby('cust_id')['date'].max().reset_index()
max_doc.rename(columns={'date':'max_date'},inplace=True)
doc_merged = doc_merged.merge(max_doc, on='cust_id', how='left')
prev_list = doc_merged[doc_merged['max_date']>doc_merged['date']]['alert_key'].tolist()

In [66]:
alert_dp = dp[dp['cust_id'].isin(cdtx[cdtx['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_dp = dp[~dp['cust_id'].isin(cdtx[cdtx['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_dp['y'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
sar_idx = train_dp[train_dp['cust_id'].isin(cdtx[cdtx['alert_key'].isin(y[y['sar_flag']==1]['alert_key'].tolist())]['cust_id'].tolist())].index
train_dp.loc[sar_idx, 'y'] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [68]:
#Data Cleansing
#SAR戶僅留前X天
def sar_pred_d(x, day):
  prev_d = x.groupby('cust_id')['tx_date'].max() - day
  prev_d = prev_d.reset_index()
  prev_d.rename(columns={'tx_date':'prev_d'}, inplace=True)
  x = x.merge(prev_d, on='cust_id', how='left')
  x.drop(x[x['tx_date']<x['prev_d']].index,inplace=True)
  x.pop('prev_d')
  return x

train_dp = sar_pred_d(train_dp, 30)
alert_dp = sar_pred_d(alert_dp, 30)


In [69]:
#特徵前處理
def preprocess(data):
  dict1 = {}
  idx = 0
  num = 0
  for i in range(0,395,1):
    dict1[i] = str(idx)
    num += 1
    if num == 7:
      idx += 1
      num = 0
  data['tx_date_group'] = data.tx_date.map(lambda x: dict1[x])
  data['session_cust_id'] = data.tx_date_group + data.cust_id
  data['date_cust_id'] = data.tx_date.astype(str) + data.cust_id
  data['date_time_cust_id'] = data.tx_date.astype(str) + data.tx_date.astype(str) + data.cust_id
  return data

train_dp = preprocess(train_dp)
alert_dp = preprocess(alert_dp)


In [70]:
#session 交易差額比率
def amt_diff_func(data):
  session_amt_diff = data.groupby(['session_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='session_cust_id', columns='debit_credit', values='tx_amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['session_amt_diff_ratio'] = \
    abs(session_amt_diff['CR'] - session_amt_diff['DB']) / abs(session_amt_diff['CR'] + session_amt_diff['DB'])
  session_amt_diff = session_amt_diff.reset_index()[['session_cust_id','session_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='session_cust_id', how='left')
#當日 交易差額比率
  date_amt_diff = data.groupby(['date_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  date_amt_diff = pd.pivot_table(date_amt_diff, index='date_cust_id', columns='debit_credit', values='tx_amt')
  date_amt_diff.fillna(1, inplace=True)
  date_amt_diff['date_amt_diff_ratio'] = \
  abs(date_amt_diff['CR'] - date_amt_diff['DB']) / abs(date_amt_diff['CR'] + date_amt_diff['DB'])
  date_amt_diff = date_amt_diff.reset_index()[['date_cust_id','date_amt_diff_ratio']]
  data = data.merge(date_amt_diff, on=['date_cust_id'], how='left')
#當時 交易差額比率
  date_time_amt_diff = data.groupby(['date_time_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  date_time_amt_diff = pd.pivot_table(date_time_amt_diff, index='date_time_cust_id', columns='debit_credit', values='tx_amt')
  date_time_amt_diff.fillna(1, inplace=True)
  date_time_amt_diff['date_time_amt_diff_ratio'] = \
  abs(date_time_amt_diff['CR'] - date_time_amt_diff['DB']) / abs(date_time_amt_diff['CR'] + date_time_amt_diff['DB'])
  date_time_amt_diff = date_time_amt_diff.reset_index()[['date_time_cust_id','date_time_amt_diff_ratio']]
  data = data.merge(date_time_amt_diff, on=['date_time_cust_id'], how='left')
  return data

train_dp = amt_diff_func(train_dp)
alert_dp = amt_diff_func(alert_dp)

In [71]:
def tx_cnt_branch_func(data):
#當時交易筆數 tx_cnt_date_time
  tx_cnt_date_time = data.groupby(['cust_id','tx_date','tx_time'])['debit_credit'].count().reset_index()
  tx_cnt_date_time.rename(columns={'debit_credit':'tx_cnt_date_time'}, inplace=True)
  data = data.merge(tx_cnt_date_time, on=['cust_id','tx_date','tx_time'], how='left')
#當日交易筆數 tx_cnt_date
  tx_cnt_date = data.groupby(['cust_id','tx_date'])['debit_credit'].count().reset_index()
  tx_cnt_date.rename(columns={'debit_credit':'tx_cnt_date'}, inplace=True)
  data = data.merge(tx_cnt_date, on=['cust_id','tx_date'], how='left')
#當session交易筆數 tx_cnt_session
  tx_cnt_session = data.groupby(['session_cust_id'])['debit_credit'].count().reset_index()
  tx_cnt_session.rename(columns={'debit_credit':'tx_cnt_session'}, inplace=True)
  data = data.merge(tx_cnt_session, on=['session_cust_id'], how='left')
#當時總分行數 txbranch_day_time_cnt
  txbranch_day_time_cnt = data.groupby(['cust_id','tx_date','tx_time'])['txbranch'].count().reset_index()
  txbranch_day_time_cnt.rename(columns={'txbranch':'txbranch_day_time_cnt'}, inplace=True)
  data = data.merge(txbranch_day_time_cnt, on=['cust_id','tx_date','tx_time'], how='left')
#單日總分行數 txbranch_day_cnt
  txbranch_day_cnt = data.groupby(['cust_id','tx_date'])['txbranch'].count().reset_index()
  txbranch_day_cnt.rename(columns={'txbranch':'txbranch_day_cnt'}, inplace=True)
  data = data.merge(txbranch_day_cnt, on=['cust_id','tx_date'], how='left')
#當session總分行數 txbranch_session_cnt
  txbranch_session_cnt = data.groupby(['session_cust_id'])['txbranch'].count().reset_index()
  txbranch_session_cnt.rename(columns={'txbranch':'txbranch_session_cnt'}, inplace=True)
  data = data.merge(txbranch_session_cnt, on=['session_cust_id'], how='left')
  return data
train_dp = tx_cnt_branch_func(train_dp)
alert_dp = tx_cnt_branch_func(alert_dp)

In [72]:
def atm_txn_func(data):
#當sessionATM 佔交易數比例
  session_atm_txn_ratio = data.groupby(['session_cust_id'])['ATM'].sum().reset_index()
  session_atm_txn_ratio.rename(columns={'ATM':'session_atm_txn_ratio'}, inplace=True)
  data = data.merge(session_atm_txn_ratio, on=['session_cust_id'], how='left')
  data.session_atm_txn_ratio = data.session_atm_txn_ratio / data.tx_cnt_date
#當日ATM 佔交易數比例
  day_atm_txn_ratio = data.groupby(['cust_id','tx_date'])['ATM'].sum().reset_index()
  day_atm_txn_ratio.rename(columns={'ATM':'day_atm_txn_ratio'}, inplace=True)
  data = data.merge(day_atm_txn_ratio, on=['cust_id','tx_date'], how='left')
  data.day_atm_txn_ratio = data.day_atm_txn_ratio / data.tx_cnt_date
#當時ATM 佔交易數比例
  day_time_atm_txn_ratio = data.groupby(['cust_id','tx_date','tx_time'])['ATM'].sum().reset_index()
  day_time_atm_txn_ratio.rename(columns={'ATM':'day_time_atm_txn_ratio'}, inplace=True)
  data = data.merge(day_time_atm_txn_ratio, on=['cust_id','tx_date','tx_time'], how='left')
  data.day_time_atm_txn_ratio = data.day_time_atm_txn_ratio / data.tx_cnt_date_time
  return data
train_dp = atm_txn_func(train_dp)
alert_dp = atm_txn_func(alert_dp)

In [73]:
def cross_bank_func(data):
  #當session跨行 佔交易數比例
  session_cross_bank_ratio = data.groupby(['session_cust_id'])['cross_bank'].sum().reset_index()
  session_cross_bank_ratio.rename(columns={'cross_bank':'session_cross_bank_ratio'}, inplace=True)
  data = data.merge(session_cross_bank_ratio, on=['session_cust_id'], how='left')
  data.session_cross_bank_ratio = data.session_cross_bank_ratio / data.tx_cnt_date
  #當日跨行 佔交易數比例
  day_cross_bank_ratio = data.groupby(['cust_id','tx_date'])['cross_bank'].sum().reset_index()
  day_cross_bank_ratio.rename(columns={'cross_bank':'day_cross_bank_ratio'}, inplace=True)
  data = data.merge(day_cross_bank_ratio, on=['cust_id','tx_date'], how='left')
  data.day_cross_bank_ratio = data.day_cross_bank_ratio / data.tx_cnt_date
  #當時跨行 佔交易數比例
  day_time_cross_bank_ratio = data.groupby(['cust_id','tx_date','tx_time'])['cross_bank'].sum().reset_index()
  day_time_cross_bank_ratio.rename(columns={'cross_bank':'day_time_cross_bank_ratio'}, inplace=True)
  data = data.merge(day_time_cross_bank_ratio, on=['cust_id','tx_date','tx_time'], how='left')
  data.day_time_cross_bank_ratio = data.day_time_cross_bank_ratio / data.tx_cnt_date_time
  return data
train_dp = cross_bank_func(train_dp)
alert_dp = cross_bank_func(alert_dp)

In [74]:
#當日沖正筆數
def reversal_daily_cnt_func(data):
  reversal_daily_cnt = data[data['tx_amt']<0].groupby(['cust_id','tx_date'])['tx_amt'].count().reset_index()
  reversal_daily_cnt.rename(columns={'tx_amt':'reversal_daily_cnt'}, inplace=True)
  data = data.merge(reversal_daily_cnt, on=['cust_id','tx_date'], how='left')
  data.fillna(0, inplace=True)
  return data

train_dp = reversal_daily_cnt_func(train_dp)
alert_dp = reversal_daily_cnt_func(alert_dp)

In [98]:
remit_test_feature_v2 = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/remit_test_feature_v2.csv')
remit_test_feature_v2.rename(columns={'trans_date':'tx_date'},inplace=True)
remit_test_feature_v2 = remit_test_feature_v2[['cust_id','tx_date','over2']]
remit_test_feature_v2

,cust_id,tx_date,over2
0,a39fea9aec90969fe66a2b2b4d1b86368a2d38e8b8d4bf...,0,0
1,7e42b5dca9b28ee8e5545beb834361e90e6197d176b389...,0,0
2,a6cdf6302aead77112013168c6d546d2df3bcb551956d2...,0,0
3,1a3efa69705f611c7ef2384a715c8142e2ee801cfec9df...,0,0
4,67f8cbb64dd3d447e992b1b299e0ceed3372188e47c88e...,0,0
...,...,...,...
23901,b600c0720096c241f5aec16b096c6a353492eee8a5855a...,364,0
23902,06c5ea5ccda4dfd8839c0dec8646fb3071d2c02883ef8a...,364,0
23903,0c2dc5fedc3689abf5ff4be14fe8fea8d23d22068297c2...,364,0
23904,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,364,0


In [101]:
train_dp = train_dp.merge(remit_test_feature_v2, on=['cust_id','tx_date'], how='left')


In [102]:
train_dp.fillna(0,inplace=True)
alert_dp = alert_dp.merge(remit_test_feature_v2, on=['cust_id','tx_date'], how='left')
alert_dp.fillna(0,inplace=True)


In [97]:
train_dp.shape

(131146, 33)

In [76]:
#remit_pivot = remit.groupby(['cust_id','trans_date','trans_no'])['trade_amount_usd'].sum().reset_index().pivot_table(index=['cust_id','trans_date'],columns='trans_no',values='trade_amount_usd').reset_index()
#remit_pivot.fillna(0, inplace=True)
#remit_pivot.rename(columns={0:'remit_trans_no_0',1:'remit_trans_no_1',2:'remit_trans_no_2',3:'remit_trans_no_3',4:'remit_trans_no_4','trans_date':'tx_date'}, inplace=True)
#remit_pivot['remit_txn_sum'] = remit_pivot[['remit_trans_no_0','remit_trans_no_1','remit_trans_no_2','remit_trans_no_3','remit_trans_no_4']].sum(axis=1)
#remit_pivot['remit_trans_no_0'] = remit_pivot['remit_trans_no_0']/remit_pivot['remit_txn_sum']
#remit_pivot['remit_trans_no_1'] = remit_pivot['remit_trans_no_1']/remit_pivot['remit_txn_sum']
#remit_pivot['remit_trans_no_2'] = remit_pivot['remit_trans_no_2']/remit_pivot['remit_txn_sum']
#remit_pivot['remit_trans_no_3'] = remit_pivot['remit_trans_no_3']/remit_pivot['remit_txn_sum']
#remit_pivot['remit_trans_no_4'] = remit_pivot['remit_trans_no_4']/remit_pivot['remit_txn_sum']
#remit_txn_cnt = remit.groupby(['cust_id','trans_date'])['trade_amount_usd'].count().reset_index()
#remit_txn_cnt.rename(columns={'trade_amount_usd':'remit_txn_cnt','trans_date':'tx_date'}, inplace=True)
#remit_pivot = remit_pivot.merge(remit_txn_cnt, on=['cust_id','tx_date'], how='left')
#remit_pivot.fillna(0, inplace=True)
#remit_pivot
#test = train_dp.groupby(['cust_id','tx_date','debit_credit','y'])['tx_amt'].sum().reset_index()
#test = test[test['debit_credit']=='CR']
#test['tx_amt'] = test['tx_amt']*30
#test = test.merge(dp_amt_sum, on=['cust_id','tx_date'], how='left')
#test['diff_ratio'] = abs(test['tx_amt']-test['dp_amt_sum'])/(test['tx_amt']+test['dp_amt_sum'])
#test
#data = train_dp.merge(remit_pivot, on=['cust_id','tx_date'], how='left')
#data.fillna(0, inplace=True)
#dp_amt_sum = data.groupby(['cust_id','tx_date'])['tx_amt'].sum().reset_index()
#dp_amt_sum.rename(columns={'tx_amt':'dp_amt_sum'}, inplace=True)
#data = data.merge(dp_amt_sum, on=['cust_id','tx_date'], how='left')
#data['remit_txn_sum_dp_ratio'] = data['remit_txn_sum']/(data['remit_txn_sum']+data['dp_amt_sum'])
#data['dp_txn_sum_remit_ratio'] = data['dp_amt_sum']/(data['remit_txn_sum']+data['dp_amt_sum'])
#data
#drop remit_txn_sum dp_amt_sum

In [103]:
train_dp.columns

Index(['cust_id', 'debit_credit', 'tx_date', 'tx_time', 'tx_type', 'tx_amt',
       'exchg_rate', 'info_asset_code', 'fiscTxId', 'txbranch', 'cross_bank',
       'ATM', 'y', 'tx_date_group', 'session_cust_id', 'date_cust_id',
       'date_time_cust_id', 'session_amt_diff_ratio', 'date_amt_diff_ratio',
       'date_time_amt_diff_ratio', 'tx_cnt_date_time', 'tx_cnt_date',
       'tx_cnt_session', 'txbranch_day_time_cnt', 'txbranch_day_cnt',
       'txbranch_session_cnt', 'session_atm_txn_ratio', 'day_atm_txn_ratio',
       'day_time_atm_txn_ratio', 'session_cross_bank_ratio',
       'day_cross_bank_ratio', 'day_time_cross_bank_ratio',
       'reversal_daily_cnt', 'over2'],
      dtype='object')

In [110]:
train_col = ['cust_id','tx_date','tx_time','session_amt_diff_ratio', 'date_amt_diff_ratio',
         'tx_cnt_date','date_time_amt_diff_ratio','tx_cnt_date_time','txbranch_day_time_cnt', 'day_time_atm_txn_ratio',
       'txbranch_day_cnt', 'day_atm_txn_ratio',
       'day_cross_bank_ratio','day_time_cross_bank_ratio','reversal_daily_cnt',  #'remit_trans_no_0',
       #'remit_trans_no_1', #'remit_trans_no_2', 'remit_trans_no_3','remit_trans_no_4', 'remit_txn', 'remit_txn_cnt','dp_txn_sum_remit_ratio','dp_txn',
       #'remit_txn_sum_dp_ratio', 
       'tx_cnt_session','txbranch_session_cnt','session_atm_txn_ratio','session_cross_bank_ratio',
       'over2',
       'y']
x_col = train_col[3:-1]
train_data = train_dp[train_col]
train_data

,cust_id,tx_date,tx_time,session_amt_diff_ratio,date_amt_diff_ratio,tx_cnt_date,date_time_amt_diff_ratio,tx_cnt_date_time,txbranch_day_time_cnt,day_time_atm_txn_ratio,...,day_atm_txn_ratio,day_cross_bank_ratio,day_time_cross_bank_ratio,reversal_daily_cnt,tx_cnt_session,txbranch_session_cnt,session_atm_txn_ratio,session_cross_bank_ratio,over2,y
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,362,15,0.516453,0.516453,3,0.516453,2,0,1.0,...,0.666667,0.000000,0.000000,0.0,3,0,0.666667,0.000000,0.0,0
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,364,14,0.999576,0.999576,1,0.999576,1,0,0.0,...,0.000000,0.000000,0.000000,0.0,1,0,0.000000,0.000000,0.0,0
2,0015cc01f553512007705edf855baa16f1b57a16fb107e...,385,11,0.741022,0.766437,3,0.766437,3,0,1.0,...,1.000000,0.000000,0.000000,0.0,4,0,1.000000,0.000000,0.0,0
3,0015cc01f553512007705edf855baa16f1b57a16fb107e...,362,15,0.516453,0.516453,3,0.516453,2,0,1.0,...,0.666667,0.000000,0.000000,0.0,3,0,0.666667,0.000000,0.0,0
4,0015cc01f553512007705edf855baa16f1b57a16fb107e...,362,11,0.516453,0.516453,3,0.516453,1,0,0.0,...,0.666667,0.000000,0.000000,0.0,3,0,0.666667,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131149,fff94defcc33daab190e702926d583f6d805294354addb...,364,14,0.812262,0.998745,4,0.998745,1,1,1.0,...,1.000000,0.250000,1.000000,0.0,12,10,3.000000,1.000000,0.0,0
131150,fff94defcc33daab190e702926d583f6d805294354addb...,382,19,0.199659,0.504529,4,0.504529,2,2,1.0,...,0.750000,0.250000,0.500000,0.0,13,11,3.000000,1.000000,0.0,0
131151,fff94defcc33daab190e702926d583f6d805294354addb...,389,23,0.606456,0.792777,9,0.792777,5,5,1.0,...,1.000000,0.333333,0.200000,0.0,19,16,1.888889,0.555556,0.0,0
131152,fff94defcc33daab190e702926d583f6d805294354addb...,383,18,0.199659,0.166769,9,0.166769,3,3,1.0,...,1.000000,0.333333,0.333333,0.0,13,11,1.333333,0.444444,0.0,0


In [111]:
Y = train_data['y']
X = train_data[x_col]

In [112]:
#model training 1
#逐筆交易處理
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#previous 30days label of 1
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 73.76%
#0.9921875
#0.5869618097786427

F1 score: 73.76%
0.9921875
0.5869618097786427


In [ ]:
F1 score: 73.76%
0.9921875
0.5869618097786427

In [113]:
pd.DataFrame({'columns':list(X.columns),'score':model.feature_importances_})

,columns,score
0,session_amt_diff_ratio,0.037580
1,date_amt_diff_ratio,0.053625
2,tx_cnt_date,0.502408
3,date_time_amt_diff_ratio,0.000000
4,tx_cnt_date_time,0.057733
5,txbranch_day_time_cnt,0.012693
6,day_time_atm_txn_ratio,0.000000
7,txbranch_day_cnt,0.015001
8,day_atm_txn_ratio,0.063538
9,day_cross_bank_ratio,0.040449


In [93]:
remit_test_feature_v2['cust_id'].value_counts()

04ac4e9d22843711c4c1276de3b55c377016e2158f280f1281ca3b17b018e994    236
124b541310713e0101c185b514ffe09ad0740ec00a23b6fb6ec47bd3e5e1ecb8    164
08d2b4ae3c03515cac79354b8f30e3287a8d427f8af2d6ece78074fa45c148d0    145
270d4e5a6bc68ead954394fc9394fa36f01a5bb258ac9fa9a581ce8957b310ae    139
5deb26d41f21535fe62ba7b458c8a1c19aa2c449c19b58e57f517d00eb27087e    138
                                                                   ... 
7a3d96671b4d5ac09a1a0f86a6d5926d0482e5a00970f6395635eb4f434ed79e      1
678502bc0eedba5a9ce3b4f1335c5137eb9e900600864900890211f95b2aef99      1
fdef227283bb6d317005a2fd944948d365c0009242701e5c2d66011686f27fd9      1
26eeff4dcdbf9ddb3e2628f8710a1e87b1ea53d37e0331c96d2b0c8536cbf132      1
67a8baafafecb5408944a7d089cda0168967144c1bd41f3380202c390430239e      1
Name: cust_id, Length: 7264, dtype: int64

In [114]:
#彙整model training 1偵測結果
#predict_proba以0.05機率區間為一個欄位判斷分佈
def mapping(proba):
    if proba <= 0.05:
        return 1
    elif 0.05 < proba <= 0.1:
        return 2
    elif 0.1 < proba <= 0.15:
        return 3
    elif 0.15 < proba <= 0.2:
        return 4
    elif 0.2 < proba <= 0.25:
        return 5
    elif 0.25 < proba <= 0.3:
        return 6
    elif 0.3 < proba <= 0.35:
        return 7
    elif 0.35 < proba <= 0.4:
        return 8
    elif 0.4 < proba <= 0.45:
        return 9
    elif 0.45 < proba <= 0.5:
        return 10
    elif 0.5 < proba <= 0.55:
        return 11
    elif 0.55 < proba <= 0.6:
        return 12
    elif 0.6 < proba <= 0.65:
        return 13
    elif 0.65 < proba <= 0.7:
        return 14
    elif 0.7 < proba <= 0.75:
        return 15
    elif 0.75 < proba <= 0.8:
        return 16
    elif 0.8 < proba <= 0.85:
        return 17
    elif 0.85 < proba <= 0.9:
        return 18
    elif 0.9 < proba <= 0.95:
        return 19
    elif 0.95 < proba <= 1:
        return 20

In [115]:
train_data['proba'] = model.predict_proba(X)[:,1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [116]:
result = train_data[['cust_id','proba','y']]
result['level'] = result["proba"].map(mapping)
result = result[['cust_id', 'level', 'y']]
result = result.groupby(['cust_id','level']).count().reset_index().pivot_table(index='cust_id', columns='level', values='y')
result.fillna(0, inplace=True)
result = result.div(result.sum(axis=1), axis=0)
result = result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(result.columns[1:])):
  result[col] = 0.0
result = result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
result = result.merge(train_data[['cust_id','y']].drop_duplicates(), on='cust_id', how='left')
result.pop('cust_id')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0       0015cc01f553512007705edf855baa16f1b57a16fb107e...
1       002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...
2       002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...
3       0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...
4       0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...
                              ...                        
5485    ffae7fef923c78298cd647a3d573e48cc1404b07ac794a...
5486    ffdbc1a5135cb6feddf7355e5656abf6973f21639c7b9b...
5487    ffdd3f98b841d4d0ef58ad0d63abddac83ffb5babf973c...
5488    ffeb846f2294ec4397f800736dc1e14eb754da5425398d...
5489    fff94defcc33daab190e702926d583f6d805294354addb...
Name: cust_id, Length: 5490, dtype: object

In [117]:
#model training 2
#歸戶判斷是否報SAR
result_col = list(result.columns)
result_col.remove('y')
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(result[result_col], result['y'], test_size=test_size, random_state=seed)
model_2nd = XGBClassifier()
model_2nd.fit(X_train, y_train)
y_pred = model_2nd.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 36.36%
#0.7142857142857143
#0.24390243902439024

F1 score: 36.36%
0.7142857142857143
0.24390243902439024


In [118]:
#Submission
alert_x = alert_dp[x_col]
alert_data = alert_dp
alert_data['proba'] = model.predict_proba(alert_x)[:,1]
alert_result = alert_data[['cust_id','proba']]
alert_result['level'] = alert_result["proba"].map(mapping)
alert_result = alert_result[['cust_id', 'level']]
alert_result['cnt'] = 1
alert_result = alert_result.groupby(['cust_id','level'])['cnt'].count().reset_index().pivot_table(index='cust_id', columns='level', values='cnt')
alert_result.fillna(0, inplace=True)
alert_result = alert_result.div(alert_result.sum(axis=1), axis=0)
alert_result = alert_result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(alert_result.columns[1:])):
  alert_result[col] = 0.0
alert_result = alert_result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
#alert_result.pop('cust_id')
alert_pred = model_2nd.predict_proba(alert_result[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
# evaluate predictions
final = alert_result[['cust_id']]
final['probability'] = alert_pred[:,1]
final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


level,cust_id,probability
0,0172056578071e83399216fcd640bdc4de4583149d0fd9...,0.022044
1,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,0.017323
2,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,0.022218
3,01c705b457297805c1c08137cb3adb164b8f393c265db2...,0.225544
4,022bec0a12b56285bf12c49ba1512c0acdbf4176a30d96...,0.022044
...,...,...
701,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,0.967491
702,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,0.022044
703,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,0.019867
704,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,0.080601


In [119]:
final = alert_time.merge(cdtx[['alert_key', 'cust_id']].merge(final, on='cust_id'), on='alert_key', how='left')[['alert_key', 'probability']]

In [120]:
final.fillna(0, inplace=True)
final

,alert_key,probability
0,352342,0.530423
1,352866,0.022044
2,352696,0.604659
3,352330,0.022218
4,352683,0.972794
...,...,...
1840,364472,0.091136
1841,364788,0.012491
1842,364673,0.422041
1843,364626,0.101568


In [121]:
doc = doc[['alert_key']]
doc

,alert_key
0,357307
1,376329
2,373644
3,357668
4,354443
...,...
3845,364485
3846,363155
3847,368710
3848,358067


In [122]:
final = doc.merge(final, on='alert_key', how='left')
final.fillna(0,inplace=True)
final

,alert_key,probability
0,357307,0.022874
1,376329,0.000000
2,373644,0.000000
3,357668,0.017323
4,354443,0.604659
...,...,...
3845,364485,0.019867
3846,363155,0.730630
3847,368710,0.000000
3848,358067,0.331423


In [123]:
final.loc[final[final['alert_key'].isin(prev_list)].index,'probability']=0

In [124]:
final.to_csv('/content/drive/MyDrive/訓練資料集_first/submission.csv')

In [348]:
final[final['probability']>0]

,alert_key,probability
0,357307,0.020106
3,357668,0.012658
5,361128,0.951214
6,364753,0.447786
9,360609,0.222253
...,...,...
3820,356610,0.069428
3832,357474,0.022766
3841,353051,0.012658
3844,360534,0.024783
